# AI Applications in Information Security - Project
## Our work is based on the following work:
### Tan, H., Luo, Q., Li, J., & Zhang, Y. (2024, March 8). LLM4Decompile: Decompiling Binary Code with Large Language Models. ArXiv.org. https://doi.org/10.48550/arXiv.2403.05286


In [3]:
import tempfile
import os
import subprocess
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
import shutil
import pandas as pd
import gc

/jet/home/ramadasa/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### LLM4Decompile-End
Direct decompilation of c assembly code using LLM4Decompile v1.5 model.

### Evaluation Function.


In [5]:
# This function creates a C file which will be given as input to another function for further processing
def create_c_file(c_func_json, output_filename='test.c'):
    try:
        includes = []
        func_body = []

        for line in c_func_json.split('\n'):
            if line.strip().startswith('#include'):
                includes.append(line)
            else:
                func_body.append(line)

        file_content = '\n'.join(includes) + '\n' + '\n'.join(func_body)

        with open(output_filename, 'w') as f:
            f.write(file_content)

        print(f"C file created successfully: {output_filename}")

    except Exception as e:
        print(f"Error creating C file: {str(e)}")

In [6]:
# This function outputs ASM in text
def get_asm(root_dir, c_file_name, opt="O1"):
  func_name = 'func0' # IMPORTANT NOTE: Change the function name if the function name is different in the .c file

  output_file = c_file_name +'_' + opt
  compile_command = f'gcc -c {os.path.join(root_dir, c_file_name)}.c -o {os.path.join(os.getcwd(), output_file)}.o -{opt} -lm' # Compiling the code
  subprocess.run(compile_command, shell=True, check=True)
  compile_command = f'objdump -d {output_file}.o > {output_file}.s' # Disassembling the binary file into assembly instructions
  subprocess.run(compile_command, shell=True, check=True)

  with open(output_file+'.s') as f:
      asm= f.read()
      if '<'+func_name+'>:' not in asm:
          raise ValueError("compile fails")
      asm = '<'+func_name+'>:' + asm.split('<'+func_name+'>:')[-1].split('\n\n')[0]
      asm_clean = ""
      asm_sp = asm.split("\n")
      for tmp in asm_sp:
        if len(tmp.split("\t"))<3 and '00' in tmp:
            continue
        idx = min(len(tmp.split("\t")) - 1, 2)
        tmp_asm = "\t".join(tmp.split("\t")[idx:])  # Removing the binary code
        tmp_asm = tmp_asm.split("#")[0].strip()  # Removing the comments
        asm_clean += tmp_asm + "\n"
      input_asm = asm_clean.strip()
      return input_asm

In [7]:
def evaluate_func(c_func, c_test, c_func_decompile):
    timeout = 10
    flag_compile = 0
    flag_run = 0
    c_include = ""
    for line in c_func.split("\n"):
        if "#include" in line:
            c_include += line + "\n"
            c_func = c_func.replace(line, "")
    for line in c_test.split("\n"):
        if "#include" in line:
            c_include += line + "\n"
            c_test = c_test.replace(line, "")
    c_combine = c_include + "\n" + c_func_decompile + "\n" + c_test
    c_onlyfunc = c_include + "\n" + c_func_decompile

    with tempfile.TemporaryDirectory() as temp_dir:
        pid = os.getpid()
        c_file = os.path.join(temp_dir, f"combine_{pid}.c")
        executable = os.path.join(temp_dir, f"combine_{pid}")
        c_file_onlyfunc = os.path.join(temp_dir, f"onlyfunc_{pid}.c")
        executable_onlyfunc = os.path.join(temp_dir, f"onlyfunc_{pid}")

        with open(c_file, "w") as f:
            f.write(c_combine)
        with open(c_file_onlyfunc, "w") as f:
            f.write(c_onlyfunc)

        # Compile the C program to an assembly
        compile_command = [
            "gcc",
            "-S",
            c_file_onlyfunc,
            "-o",
            executable_onlyfunc,
            "-lm",
        ]
        try:
            subprocess.run(compile_command, check=True, timeout=timeout)
            flag_compile = 1
        except:
            return flag_compile, flag_run

        # Compile the C program to an executable
        compile_command = ["gcc", c_file, "-o", executable, "-lm"]
        try:
            subprocess.run(compile_command, check=True, timeout=timeout)
            flag_compile = 1
        except:
            return flag_compile, flag_run

        # Run the compiled executable
        run_command = [executable]
        try:
            process = subprocess.run(
                run_command, capture_output=True, text=True, timeout=timeout, check=True
            )
            flag_run = 1
        except:
            if "process" in locals() and process:
                process.kill()
                process.wait()
            return flag_compile, flag_run

    return flag_compile, flag_run


In [10]:
def eval_llm(test_file_path, eval=True):
  root_dir = os.getcwd()
  output_folder_path = os.path.join(root_dir, "llm_outputs")
  os.makedirs(output_folder_path, exist_ok=True)

  optimizations = ["O0", "O1"]
  before = "# This is the assembly code:\n"
  after = "\n# What is the source code?\n"
  with open(test_file_path, "r") as f:
    test_set = json.load(f)
  for i in test_set:
    c_file_name = str(i["task_id"])
    create_c_file(i["c_func"], c_file_name+".c")
    for opt in optimizations:
      asm = get_asm(root_dir, c_file_name, opt)
      prompt = before + asm.strip() + after
      print(prompt)
      inputs = tokenizer(prompt, return_tensors="pt").to(llm4decompile_end_model.device)
      with torch.no_grad():
        outputs = llm4decompile_end_model.generate(**inputs, max_new_tokens=2048)
      c_func_decompile = tokenizer.decode(outputs[0][len(inputs[0]) : -1])
      print(c_func_decompile)
      create_c_file(c_func_decompile,str(i["task_id"])+"_"+opt+".c")
      shutil.move(os.path.join(os.getcwd(), str(i["task_id"])+"_"+opt+".c"), os.path.join(output_folder_path, str(i["task_id"])+"_"+opt+".c"))

      if eval==True:
        flag_compile, flag_run = evaluate_func(i["c_func"],i["c_test"],c_func_decompile)
        new_row = {'Task ID': i["task_id"], 'Optimization': opt, 'Compilability': flag_compile, 'Re-executability': flag_run}
        df.loc[len(df)] = new_row
        print(new_row)
        print("\n\n\n")

      torch.cuda.empty_cache()
      gc.collect()

    shutil.move(os.path.join(os.getcwd(), c_file_name+".c"), os.path.join(output_folder_path, c_file_name+".c"))

In [9]:
model_path = 'LLM4Binary/llm4decompile-6.7b-v1.5' # V1.5 Model
tokenizer = AutoTokenizer.from_pretrained(model_path)
llm4decompile_end_model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16).cuda()

local_save_path = '/local/llm4decompile-6.7b-v1.5'
llm4decompile_end_model.save_pretrained(local_save_path)
tokenizer.save_pretrained(local_save_path)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  5.24it/s]


('/local/llm4decompile-6.7b-v1.5/tokenizer_config.json',
 '/local/llm4decompile-6.7b-v1.5/special_tokens_map.json',
 '/local/llm4decompile-6.7b-v1.5/tokenizer.json')

In [11]:
df = pd.DataFrame(columns=['Task ID', 'Optimization', 'Compilability', 'Re-executability'])
eval_llm("dataset.json", eval=True)

Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


C file created successfully: 1.c
# This is the assembly code:
<func0>:
push   %rbp
mov    %rsp,%rbp
mov    %rdi,-0x18(%rbp)
mov    %rsi,-0x20(%rbp)
mov    -0x18(%rbp),%rax
mov    (%rax),%eax
mov    %eax,-0x4(%rbp)
mov    -0x20(%rbp),%rax
mov    (%rax),%edx
mov    -0x18(%rbp),%rax
mov    %edx,(%rax)
mov    -0x20(%rbp),%rax
mov    -0x4(%rbp),%edx
mov    %edx,(%rax)
nop
pop    %rbp
retq
# What is the source code?

void func0(int *a, int *b)
{
  int c;
  c = *a;
  *a = *b;
  *b = c;
}
C file created successfully: 1_O0.c
{'Task ID': 1, 'Optimization': 'O0', 'Compilability': 1, 'Re-executability': 1}






Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


# This is the assembly code:
<func0>:
mov    (%rdi),%eax
mov    (%rsi),%edx
mov    %edx,(%rdi)
mov    %eax,(%rsi)
retq
# What is the source code?

void func0(int *a, int *b)
{
 int tmp;

 tmp = *a;
 *a = *b;
 *b = tmp;
}
C file created successfully: 1_O1.c
{'Task ID': 1, 'Optimization': 'O1', 'Compilability': 1, 'Re-executability': 1}






Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


...

/var/tmp/tmpoxuftcv1/onlyfunc_56483.c: In function ‘func0’:
/var/tmp/tmpoxuftcv1/onlyfunc_56483.c:18:3: warning: implicit declaration of function ‘print_int’; did you mean ‘printf’? [-Wimplicit-function-declaration]
   print_int(0);
   ^~~~~~~~~
   printf
/var/tmp/tmpoxuftcv1/combine_56483.c: In function ‘func0’:
/var/tmp/tmpoxuftcv1/combine_56483.c:18:3: warning: implicit declaration of function ‘print_int’; did you mean ‘printf’? [-Wimplicit-function-declaration]
   print_int(0);
   ^~~~~~~~~
   printf
In file included from /var/tmp/tmpoxuftcv1/combine_56483.c:3:
/var/tmp/tmpoxuftcv1/combine_56483.c: In function ‘main’:
/var/tmp/tmpoxuftcv1/combine_56483.c:39:12: warning: implicit declaration of function ‘strstr’ [-Wimplicit-function-declaration]
     assert(strstr(buffer, "Product Matrix:\n19 22 \n43 50 \n") != NULL);
            ^~~~~~
/var/tmp/tmpoxuftcv1/combine_56483.c:39:12: warning: incompatible implicit declaration of built-in function ‘strstr’
/var/tmp/tmpoxuftcv1/combine_56

C file created successfully: 51.c
# This is the assembly code:
<func0>:
push   %rbp
mov    %rsp,%rbp
sub    $0x30,%rsp
movl   $0x1,-0x20(%rbp)
movl   $0x2,-0x1c(%rbp)
movl   $0x3,-0x18(%rbp)
movl   $0x4,-0x14(%rbp)
movl   $0x0,-0x4(%rbp)
jmp    6f <func0+0x6f>
movl   $0x0,-0x8(%rbp)
jmp    65 <func0+0x65>
mov    -0x8(%rbp),%eax
cltq
mov    -0x4(%rbp),%edx
movslq %edx,%rdx
add    %rdx,%rdx
add    %rdx,%rax
mov    -0x20(%rbp,%rax,4),%eax
mov    -0x4(%rbp),%edx
movslq %edx,%rdx
mov    -0x8(%rbp),%ecx
movslq %ecx,%rcx
add    %rcx,%rcx
add    %rcx,%rdx
mov    %eax,-0x30(%rbp,%rdx,4)
addl   $0x1,-0x8(%rbp)
cmpl   $0x1,-0x8(%rbp)
jle    36 <func0+0x36>
addl   $0x1,-0x4(%rbp)
cmpl   $0x1,-0x4(%rbp)
jle    2d <func0+0x2d>
mov    $0x0,%edi
callq  7f <func0+0x7f>
movl   $0x0,-0xc(%rbp)
jmp    cf <func0+0xcf>
movl   $0x0,-0x10(%rbp)
jmp    bb <func0+0xbb>
mov    -0x10(%rbp),%eax
cltq
mov    -0xc(%rbp),%edx
movslq %edx,%rdx
add    %rdx,%rdx
add    %rdx,%rax
mov    -0x30(%rbp,%rax,4),%eax
mov    %ea

/var/tmp/tmp8zdlfz99/onlyfunc_56483.c: In function ‘func0’:
/var/tmp/tmp8zdlfz99/onlyfunc_56483.c:15:3: warning: implicit declaration of function ‘func1’; did you mean ‘func0’? [-Wimplicit-function-declaration]
   func1(0);
   ^~~~~
   func0
/var/tmp/tmp8zdlfz99/combine_56483.c: In function ‘func0’:
/var/tmp/tmp8zdlfz99/combine_56483.c:15:3: warning: implicit declaration of function ‘func1’; did you mean ‘func0’? [-Wimplicit-function-declaration]
   func1(0);
   ^~~~~
   func0
In file included from /var/tmp/tmp8zdlfz99/combine_56483.c:3:
/var/tmp/tmp8zdlfz99/combine_56483.c: In function ‘main’:
/var/tmp/tmp8zdlfz99/combine_56483.c:37:12: warning: implicit declaration of function ‘strstr’ [-Wimplicit-function-declaration]
     assert(strstr(buffer, "Transpose Matrix:\n1 3 \n2 4 \n") != NULL);
            ^~~~~~
/var/tmp/tmp8zdlfz99/combine_56483.c:37:12: warning: incompatible implicit declaration of built-in function ‘strstr’
/var/tmp/tmp8zdlfz99/combine_56483.c:37:12: note: include ‘<s

# This is the assembly code:
<func0>:
sub    $0x8,%rsp
mov    $0x0,%edi
callq  e <func0+0xe>
mov    $0x1,%esi
mov    $0x0,%edi
mov    $0x0,%eax
callq  22 <func0+0x22>
mov    $0x3,%esi
mov    $0x0,%edi
mov    $0x0,%eax
callq  36 <func0+0x36>
mov    $0xa,%edi
callq  40 <func0+0x40>
mov    $0x2,%esi
mov    $0x0,%edi
mov    $0x0,%eax
callq  54 <func0+0x54>
mov    $0x4,%esi
mov    $0x0,%edi
mov    $0x0,%eax
callq  68 <func0+0x68>
mov    $0xa,%edi
callq  72 <func0+0x72>
add    $0x8,%rsp
retq
# What is the source code?

void func0()
{
  int a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z;

  a = func1(0);
  b = func1(1);
  c = func1(2);
  d = func1(3);
  e = func1(4);
  f = func1(5);
  g = func1(6);
  h = func1(7);
  i = func1(8);
  j = func1(9);
  k = func1(10);
  l = func1(11);
  m = func1(12);
  n = func1(13);
  o = func1(14);
  p = func1(15);
  q = func1(16);
  r = func1(17);
  s = func1(18);
  t = func1(19);
  u = func1(20);
  v = func1(21);
  w = func1(22);


/var/tmp/tmp1lpn8io7/onlyfunc_56483.c: In function ‘func0’:
/var/tmp/tmp1lpn8io7/onlyfunc_56483.c:9:7: warning: implicit declaration of function ‘func1’; did you mean ‘func0’? [-Wimplicit-function-declaration]
   a = func1(0);
       ^~~~~
       func0
/var/tmp/tmp1lpn8io7/onlyfunc_56483.c:36:3: warning: implicit declaration of function ‘func2’; did you mean ‘func0’? [-Wimplicit-function-declaration]
   func2(a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z);
   ^~~~~
   func0
/var/tmp/tmp1lpn8io7/combine_56483.c: In function ‘func0’:
/var/tmp/tmp1lpn8io7/combine_56483.c:9:7: warning: implicit declaration of function ‘func1’; did you mean ‘func0’? [-Wimplicit-function-declaration]
   a = func1(0);
       ^~~~~
       func0
/var/tmp/tmp1lpn8io7/combine_56483.c:36:3: warning: implicit declaration of function ‘func2’; did you mean ‘func0’? [-Wimplicit-function-declaration]
   func2(a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z);
 

In [14]:
df

,Task ID,Optimization,Compilability,Re-executability
0,1,O0,1,1
1,1,O1,1,1
2,2,O0,1,0
3,2,O1,0,0
4,3,O0,0,0
...,...,...,...,...
101,49,O1,1,0
102,50,O0,1,0
103,50,O1,1,0
104,51,O0,1,0


In [22]:
df['Compilability'].sum()

98

In [23]:
df['Re-executability'].sum()

2

In [24]:
df.to_csv("llm4decompile_eval.csv")

In [25]:
!rm *.c
!rm *.s
!rm *.o

rm: cannot remove '*.c': No such file or directory


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [56]:
# rm -rf llm_outputs

### LLM4Decompile-Ref
Refine the Ghrdra reverse code using LLM4Decompile v2 model.


In [6]:
# model_path = 'LLM4Binary/llm4decompile-1.3b-v2' # V2 Model for Refine
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# llm4decompile_ref_model = AutoModelForCausalLM.from_pretrained(model_path,torch_dtype=torch.bfloat16).cuda()

In [7]:
# with open(
#     "/content/LLM4Decompile/decompile-eval/decompile-eval-executable-gcc-ghidra.json",
#     "r",
# ) as f:
#     ref_sample = json.load(f)[0]


# # run the generate
# before = "# This is the assembly code:\n"
# after = "\n# What is the source code?\n"
# prompt = before + ref_sample["input_asm_prompt"].strip() + after
# print(prompt)
# print("-----------------")

# inputs = tokenizer(prompt, return_tensors="pt").to(llm4decompile_ref_model.device)
# with torch.no_grad():
#     outputs = llm4decompile_ref_model.generate(**inputs, max_new_tokens=2048)

# c_func_decompile = tokenizer.decode(outputs[0][len(inputs[0]) : -1])
# print(c_func_decompile)
# print("-----------------")

# # run the test
# c_func, c_test = ref_sample["c_func"], ref_sample["c_test"]
# flag_compile, flag_run = evaluate_func(c_func, c_test, c_func_decompile)
# print("Compile:", flag_compile)
# print("Run:", flag_run)
